In [ ]:
import os
import numpy as np
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.layers as layers
from text_loader import TextLoader

In [ ]:
num_layers  = 3
hidden_size = 512
batch_size  = 200
max_length  = 30
learning_rate = 0.001

loader = TextLoader("sentences_for_word2vec.txt")
vocab_size = len(loader.vocab)

In [ ]:
# with tf.device("/gpu:5"):
X = tf.placeholder(tf.float32, [None, max_length, 100])
y = tf.placeholder(tf.int32, [None, max_length]) # 인풋은 워드투벡 아웃풋은 단어?

y_one_hot = tf.one_hot(y, vocab_size)

cells = [rnn.BasicLSTMCell(hidden_size) for _ in range(num_layers)]
cells = rnn.MultiRNNCell(cells, state_is_tuple=True)

initial_state = cells.zero_state(batch_size, tf.float32)
outputs, _ = tf.nn.dynamic_rnn(cells, X, initial_state=initial_state, dtype=tf.float32)

outputs = tf.reshape(outputs, [-1, hidden_size])
logits = layers.linear(outputs, vocab_size)
y_flat = tf.reshape(y_one_hot, [-1, vocab_size])


loss_op = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_flat)
loss_op = tf.reduce_mean(loss_op)

opt = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

y_softmax = tf.nn.softmax(logits)
pred = tf.argmax(y_softmax, axis=1)
pred = tf.reshape(pred, [batch_size, -1])

In [ ]:
saver = tf.train.Saver()

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
# model_filepath=

with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "char-rnn_230000") # 

    for step in range(230000, 1000000):
        X_wv, X_idx, y_wv, y_idx = loader.next_batch(batch_size, max_length)
        loss, _ = sess.run([loss_op, opt], feed_dict={X: X_wv, y: y_idx})

        if (step+1) % 10 == 0:
            print("{:08d} step, loss:{:.4f}".format(step+1, loss))

            random = np.random.randint(0, batch_size)
            results = sess.run(pred, feed_dict={X: X_wv})
            words = [loader.words[word] for word in results[random]]
            print(" ".join(words))

        if (step+1) % 1000 == 0: 
            saver.save(sess, "char-rnn_"+str(step+1)) # 원래 checkpoints/